https://colab.research.google.com/drive/1P9iQHtUH5KUbTVtp8B4-AopZzEEPE0lw?usp=sharing#scrollTo=4YS7XI2bZTyz

https://github.com/aub-mind/arabert

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install optuna==2.3.0
!pip -q install transformers==4.2.1
!pip -q install farasapy
!pip -q install pyarabic
!git clone https://github.com/aub-mind/arabert

     |████████████████████████████████| 258 kB 7.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 164 kB 56.1 MB/s 
     |████████████████████████████████| 80 kB 12.5 MB/s 
     |████████████████████████████████| 75 kB 5.9 MB/s 
     |████████████████████████████████| 141 kB 64.8 MB/s 
     |████████████████████████████████| 111 kB 74.8 MB/s 
     |████████████████████████████████| 49 kB 8.5 MB/s 
     |████████████████████████████████| 1.8 MB 8.2 MB/s 
     |████████████████████████████████| 2.9 MB 60.1 MB/s 
     |████████████████████████████████| 895 kB 74.5 MB/s 
     |████████████████████████████████| 126 kB 7.3 MB/s 
Cloning into 'arabert'...
remote: Enumerating objects: 544, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 544 (delta 175), reused 237 (delta 87), pack-reused 214
Receiving

In [3]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [4]:
hard_map = { 0 : 'fake', 1 : 'real', }

df_train = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/train_news.csv", header=0,encoding='utf-8')
df_train.label= df_train.label.apply(lambda x: hard_map[x])

df_val = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/val_news.csv", header=0,encoding='utf-8')
df_val.label= df_val.label.apply(lambda x: hard_map[x])

label_list_HARD = ['fake', 'real']


# Trainer

In [5]:
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures


In [6]:
#https://huggingface.co/aubmindlab/aragpt2-mega-detector-long
dataset_name = 'NEWS'
model_name = 'aubmindlab/aragpt2-mega-detector-long' 
task_name = 'classification'
max_len = 414

In [7]:
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

df_train.text = df_train.text.apply(lambda x:   arabert_prep.preprocess(x))
df_val.text = df_val.text.apply(lambda x:   arabert_prep.preprocess(x))  

In [8]:
df_val.text

0       أطاحت الروسية داريا كاساتكينا بالفرنسية أليز ك...
1       قالت مصادر أمنية بشمال سيناء إن شرطيا مصريا قت...
2       عاجل قبل قليل ترامب يحس بالإغماء والشك بإصابته...
3       انا _ لله _ وإنا _ إليه _ راجعون توفيت صباح ال...
4       الفيتنامي فام دينه نوين ، وهو تاجر من مدينة هو...
                              ...                        
1277    الأوبئة تنتشر كل مئة عام ، في السنة ذات الرقم ...
1278    وزير الأوقاف : ندرس تقليل زمن الخطبة لصلاة الج...
1279    قالت صحيفة لوفيغارو إن وزارة الداخلية الفرنسية...
1280    بعد ما فاجئت العالم بإسلامها ، مريم بيترونين ت...
1281    نبه أطباء وأكاديميون بريطانيون إلى أن المشروبا...
Name: text, Length: 1282, dtype: object

In [9]:
class BERTDataset():
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [10]:
label_map = { v:index for index, v in enumerate(label_list_HARD) }
print(label_map)
train_dataset = BERTDataset(df_train.text.to_list(),df_train.label.to_list(),model_name, max_len,label_map)
val_dataset  = BERTDataset(df_val.text.to_list()   ,df_val.label.to_list()  ,model_name,  max_len, label_map)

{'fake': 0, 'real': 1}


In [11]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [12]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  print(confusion_matrix(p.label_ids,preds))

  # macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  # c_repoert= classification_report(p.label_ids,preds))

  return {
      'macro_f1' : macro_f1,
      # 'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc,
      # 'classification_report' :c_repoert
  }

# HyperParameter Search

In [13]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.lr_scheduler_type = 'cosine'
training_args.fp16 = True
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8
training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000
# training_args.save_steps = 
#training_args.eval_steps = 
training_args.disable_tqdm = True
# print("Logging Step:", training_args.logging_steps)
# print("Eval Step:",training_args.eval_steps)

In [14]:
steps_per_epoch = (len(df_train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)

160
1280


In [15]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset, 
    eval_dataset=val_dataset, 
    model_init=model_init,
    compute_metrics=compute_metrics,
)

# Regular Training

In [16]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
# training_args.per_device_train_batch_size =16
# training_args.per_device_eval_batch_size = 16

training_args.per_device_train_batch_size = 16  
training_args.per_device_eval_batch_size = 16


training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8


steps_per_epoch = (len(df_train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.load_best_model_at_end=True

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

160
1280


In [17]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
) 

In [18]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:911: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
0,No log,0.256383,0.892171,0.942960,0.861237,0.922777,16.543800,77.491000
1,No log,0.109367,0.966821,0.971855,0.962073,0.974259,16.713500,76.705000
2,No log,0.230483,0.942558,0.968582,0.922601,0.957098,16.691400,76.806000
3,0.131200,0.128593,0.975754,0.983413,0.968716,0.981279,16.729800,76.630000
4,0.131200,0.162868,0.974768,0.981896,0.968183,0.980499,16.705300,76.742000
5,0.131200,0.168303,0.972644,0.981823,0.964342,0.978939,16.707200,76.733000
6,0.005100,0.170531,0.972644,0.981823,0.964342,0.978939,16.692800,76.799000
7,0.005100,0.171239,0.972644,0.981823,0.964342,0.978939,16.703000,76.753000


              precision    recall  f1-score   support

           0       0.98      0.73      0.83       343
           1       0.91      0.99      0.95       939

    accuracy                           0.92      1282
   macro avg       0.94      0.86      0.89      1282
weighted avg       0.93      0.92      0.92      1282

[[250  93]
 [  6 933]]
              precision    recall  f1-score   support

           0       0.97      0.94      0.95       343
           1       0.98      0.99      0.98       939

    accuracy                           0.97      1282
   macro avg       0.97      0.96      0.97      1282
weighted avg       0.97      0.97      0.97      1282

[[321  22]
 [ 11 928]]


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:911: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,


              precision    recall  f1-score   support

           0       0.99      0.85      0.91       343
           1       0.95      1.00      0.97       939

    accuracy                           0.96      1282
   macro avg       0.97      0.92      0.94      1282
weighted avg       0.96      0.96      0.96      1282

[[291  52]
 [  3 936]]
              precision    recall  f1-score   support

           0       0.99      0.94      0.96       343
           1       0.98      1.00      0.99       939

    accuracy                           0.98      1282
   macro avg       0.98      0.97      0.98      1282
weighted avg       0.98      0.98      0.98      1282

[[323  20]
 [  4 935]]
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       343
           1       0.98      0.99      0.99       939

    accuracy                           0.98      1282
   macro avg       0.98      0.97      0.97      1282
weighted avg       0.98     

TrainOutput(global_step=1280, training_loss=0.053293740612571125, metrics={'train_runtime': 1649.4287, 'train_samples_per_second': 0.776, 'total_flos': 13774160747798208, 'epoch': 8.0})

In [19]:
trainer.evaluate(val_dataset)

              precision    recall  f1-score   support

           0       0.99      0.93      0.96       343
           1       0.98      1.00      0.99       939

    accuracy                           0.98      1282
   macro avg       0.98      0.96      0.97      1282
weighted avg       0.98      0.98      0.98      1282

[[320  23]
 [  4 935]]


{'epoch': 8.0,
 'eval_accuracy': 0.9789391575663027,
 'eval_loss': 0.17123939096927643,
 'eval_macro_f1': 0.972643620203604,
 'eval_macro_precision': 0.9818229851284827,
 'eval_macro_recall': 0.9643423777543879,
 'eval_runtime': 16.6726,
 'eval_samples_per_second': 76.893}

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
trainer._save("/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraGPT2")
tokenizer.save_pretrained("/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraGPT2")

('/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraGPT2/tokenizer_config.json',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraGPT2/special_tokens_map.json',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraGPT2/vocab.txt',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/AraGPT2/added_tokens.json')

# Test

In [21]:
df_test = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/test_news.csv", header=0,encoding='utf-8')

In [22]:
df_test.text = df_test.text.apply(lambda x:   arabert_prep.preprocess(x))

In [23]:
df_test.label= df_test.label.apply(lambda x: hard_map[x])
test_dataset = BERTDataset(df_test.text.to_list(),df_test.label.to_list(),model_name, max_len,label_map)


In [24]:
trainer.evaluate(test_dataset)

              precision    recall  f1-score   support

           0       0.98      0.93      0.96       400
           1       0.98      0.99      0.99      1203

    accuracy                           0.98      1603
   macro avg       0.98      0.96      0.97      1603
weighted avg       0.98      0.98      0.98      1603

[[ 373   27]
 [   7 1196]]


{'epoch': 8.0,
 'eval_accuracy': 0.9787897691827823,
 'eval_loss': 0.17072464525699615,
 'eval_macro_f1': 0.9711977085843533,
 'eval_macro_precision': 0.9797510435942678,
 'eval_macro_recall': 0.9633406068162926,
 'eval_runtime': 21.0452,
 'eval_samples_per_second': 76.169}

In [25]:
raw_pred, _, _ = trainer.predict(test_dataset)
raw_pred

              precision    recall  f1-score   support

           0       0.98      0.93      0.96       400
           1       0.98      0.99      0.99      1203

    accuracy                           0.98      1603
   macro avg       0.98      0.96      0.97      1603
weighted avg       0.98      0.98      0.98      1603

[[ 373   27]
 [   7 1196]]


array([[ 4.984, -4.742],
       [-4.637,  5.22 ],
       [ 4.95 , -4.727],
       ...,
       [-4.6  ,  5.18 ],
       [ 3.268, -3.225],
       [-4.62 ,  5.184]], dtype=float16)

# Predict

In [26]:
class Dataset():
    def __init__(self, text=None, target= None, model_name = None, max_len= None, label_map= None):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask)

In [27]:

title_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/title_example.csv")
title_example.shape

count=0
for title in title_example.text:
   test_dataset = Dataset([title],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(test_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1

    
print("Real : "+str(count))

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
Real : 3


In [28]:
text_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/text_example.csv")
text_example.shape
count=0

for text in text_example.text:
   text_dataset = Dataset([text],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(text_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1
    
print("Real : "+str(count))

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
Real : 0


In [29]:
#  ----- 3. Predict -----#
# # Load test data
# test_data = pd.("drive/My Drive/Arabic_FakeNews/Model/Data/test_news.csv", header=0,encoding='utf-8')
# X_test = list(test_data["text"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized)

# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# # Define test trainer
# test_trainer = Trainer(model)

# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset)

# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)